# Big Data Project - Snehal Thorat

In [1]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import *

"""
import pandas as pd
#from IPython.core.display import display, HTML
#display(HTML('<h1>Hello, world!</h1>'))
"""

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1557151419635_0004,pyspark3,idle,Link,Link,✔


SparkSession available as 'spark'.
"\nimport pandas as pd\n#from IPython.core.display import display, HTML\n#display(HTML('<h1>Hello, world!</h1>'))\n"

In [3]:
avgcc_inc_All_5MayDF = spark.read.parquet("s3://snehalpreprocessedfiles/Dataframes_equi_wells/avgcc_inc_All_5May.pq")

In [4]:
avgcc_inc_All_5MayDF.columns

['date_received', 'product', 'sub_product', 'issue', 'sub_issue', 'consumer_complaint_narrative', 'company_public_response', 'company', 'state', 'zip_code', 'tags', 'consumer_consent_provided', 'submitted_via', 'date_sent_to_company', 'company_response', 'timely_response', 'consumer_disputed', 'complaint_id', 'population', 'c_zipcode', 'zipcode_income', 'Total_Income', 'NUMDEP', 'zipcode_agg', 'Number_of_Complaints', 'zipcode_avgcc', 'Average_Complaints', 'Average_Income']

In [5]:
finalDF = avgcc_inc_All_5MayDF.drop('consumer_complaint_narrative','c_zipcode','zipcode_income')
finalDF.printSchema()

root
 |-- date_received: string (nullable = true)
 |-- product: string (nullable = true)
 |-- sub_product: string (nullable = true)
 |-- issue: string (nullable = true)
 |-- sub_issue: string (nullable = true)
 |-- company_public_response: string (nullable = true)
 |-- company: string (nullable = true)
 |-- state: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- consumer_consent_provided: string (nullable = true)
 |-- submitted_via: string (nullable = true)
 |-- date_sent_to_company: string (nullable = true)
 |-- company_response: string (nullable = true)
 |-- timely_response: string (nullable = true)
 |-- consumer_disputed: string (nullable = true)
 |-- complaint_id: string (nullable = true)
 |-- population: integer (nullable = true)
 |-- Total_Income: string (nullable = true)
 |-- NUMDEP: string (nullable = true)
 |-- zipcode_agg: string (nullable = true)
 |-- Number_of_Complaints: long (nullable = true)
 |-- zipcode_avgcc: str

In [11]:
finalDF.createOrReplaceTempView("combined_dataset")

In [12]:
spark.sql("show tables").show()
spark.sql("DESC combined_dataset").show(30,truncate=False)

+--------+----------------+-----------+
|database|       tableName|isTemporary|
+--------+----------------+-----------+
|        |combined_dataset|       true|
|        |            corr|       true|
+--------+----------------+-----------+

+-------------------------+---------+-------+
|col_name                 |data_type|comment|
+-------------------------+---------+-------+
|date_received            |string   |null   |
|product                  |string   |null   |
|sub_product              |string   |null   |
|issue                    |string   |null   |
|sub_issue                |string   |null   |
|company_public_response  |string   |null   |
|company                  |string   |null   |
|state                    |string   |null   |
|zip_code                 |string   |null   |
|tags                     |string   |null   |
|consumer_consent_provided|string   |null   |
|submitted_via            |string   |null   |
|date_sent_to_company     |string   |null   |
|company_response      

# a)  Find the top (20 or less) for each attributes based on the number of complaints

In [7]:
finalDF.count()

3668131

In [8]:
spark.sql("SELECT product,COUNT(product) Number_of_complaints, COUNT(product)/3668131*100 Percent_of_Total FROM combined_dataset GROUP BY product ORDER BY Number_of_complaints DESC").show(truncate=False)

+----------------------------------------------------------------------------+--------------------+--------------------+
|product                                                                     |Number_of_complaints|Percent_of_Total    |
+----------------------------------------------------------------------------+--------------------+--------------------+
|Mortgage                                                                    |895869              |24.423037236129243  |
|Debt collection                                                             |663390              |18.08523196145394   |
|Credit reporting, credit repair services, or other personal consumer reports|573972              |15.647532762597628  |
|Credit reporting                                                            |456502              |12.445084431281217  |
|Bank account or service                                                     |281522              |7.6748076881659895  |
|Credit card                    

In [9]:
spark.sql("SELECT sub_product,COUNT(sub_product) Number_of_complaints, COUNT(sub_product)/3668131*100 Percent_of_Total  FROM combined_dataset GROUP BY sub_product ORDER BY Number_of_complaints DESC").show(truncate=False)

+------------------------------------------+--------------------+------------------+
|sub_product                               |Number_of_complaints|Percent_of_Total  |
+------------------------------------------+--------------------+------------------+
|                                          |747242              |20.371191759509134|
|Credit reporting                          |565326              |15.411826894950046|
|Other mortgage                            |336071              |9.161913792064679 |
|Checking account                          |277317              |7.560171651448654 |
|Conventional fixed mortgage               |217098              |5.918490915400786 |
|I do not know                             |142432              |3.882958378531192 |
|Other (i.e. phone, health club, etc.)     |135911              |3.7051839206396937|
|Conventional adjustable mortgage (ARM)    |86358               |2.354277968807548 |
|FHA mortgage                              |86177               |

In [10]:
spark.sql("SELECT company,COUNT(company) Number_of_complaints, COUNT(company)/3668131*100 Percent_of_Total  FROM combined_dataset GROUP BY company ORDER BY Number_of_complaints DESC").show(truncate=False)
spark.sql("SELECT company_public_response,COUNT(company_public_response) Number_of_complaints, COUNT(company_public_response)/3668131*100 Percent_of_Total  FROM combined_dataset GROUP BY company_public_response ORDER BY Number_of_complaints DESC").show()

+--------------------------------------+--------------------+------------------+
|company                               |Number_of_complaints|Percent_of_Total  |
+--------------------------------------+--------------------+------------------+
|EQUIFAX, INC.                         |327716              |8.934141119823693 |
|Experian Information Solutions Inc.   |309790              |8.445445378041297 |
|BANK OF AMERICA, NATIONAL ASSOCIATION |280684              |7.651962266342179 |
|TRANSUNION INTERMEDIATE HOLDINGS, INC.|279988              |7.632988025782067 |
|WELLS FARGO & COMPANY                 |232182              |6.329708508229396 |
|JPMORGAN CHASE & CO.                  |194225              |5.294930851706224 |
|CITIBANK, N.A.                        |143621              |3.9153727061547148|
|CAPITAL ONE FINANCIAL CORPORATION     |97452               |2.656720820494143 |
|OCWEN LOAN SERVICING LLC              |91692               |2.499692622755294 |
|Navient Solutions, LLC.    

In [11]:
spark.sql("SELECT company_response,COUNT(company_response) Number_of_complaints, COUNT(company_response)/3668131*100 Percent_of_Total  FROM combined_dataset GROUP BY company_response ORDER BY Number_of_complaints DESC").show(truncate=False)

+-------------------------------+--------------------+-------------------+
|company_response               |Number_of_complaints|Percent_of_Total   |
+-------------------------------+--------------------+-------------------+
|Closed with explanation        |2812496             |76.67381562981257  |
|Closed with non-monetary relief|468068              |12.760394871393633 |
|Closed with monetary relief    |208474              |5.683384808230676  |
|Closed without relief          |70247               |1.9150624664168212 |
|Closed                         |53598               |1.4611800941678472 |
|In progress                    |21432               |0.5842757524199654 |
|Closed with relief             |19974               |0.5445279898673194 |
|Untimely response              |13830               |0.37703124561254764|
|                               |12                  |3.27142078622601E-4|
+-------------------------------+--------------------+-------------------+

In [12]:
spark.sql("SELECT consumer_consent_provided,COUNT(consumer_consent_provided) Number_of_complaints, COUNT(*)/3668131*100 Percent_of_Total  FROM combined_dataset GROUP BY consumer_consent_provided ORDER BY Number_of_complaints DESC").show(truncate=False)

+-------------------------+--------------------+------------------+
|consumer_consent_provided|Number_of_complaints|Percent_of_Total  |
+-------------------------+--------------------+------------------+
|N/A                      |2255326             |61.48433630096635 |
|Consent not provided     |1195515             |32.5919385103749  |
|Consent provided         |147228              |4.013706162620691 |
|                         |38034               |1.036876818194334 |
|Other                    |25704               |0.7007383324096114|
|Consent withdrawn        |6324                |0.1724038754341107|
+-------------------------+--------------------+------------------+

In [13]:
spark.sql("SELECT consumer_disputed,COUNT(consumer_disputed)  Number_of_complaints, COUNT(*)/3668131*100 Percent_of_Total FROM combined_dataset GROUP BY consumer_disputed ORDER BY Number_of_complaints DESC").show(truncate=False)

+-----------------+--------------------+------------------+
|consumer_disputed|Number_of_complaints|Percent_of_Total  |
+-----------------+--------------------+------------------+
|No               |1983293             |54.06821621147119 |
|N/A              |1233420             |33.62529855122404 |
|Yes              |451418              |12.306485237304775|
+-----------------+--------------------+------------------+

In [14]:
spark.sql("SELECT date_received,COUNT(date_received) Number_of_complaints, COUNT(*)/3668131*100 Percent_of_Total  FROM combined_dataset GROUP BY date_received ORDER BY Number_of_complaints DESC").show(truncate=False)

+-------------+--------------------+-------------------+
|date_received|Number_of_complaints|Percent_of_Total   |
+-------------+--------------------+-------------------+
|09/08/2017   |7236                |0.1972666734094284 |
|09/09/2017   |5952                |0.16226247099681007|
|01/19/2017   |4374                |0.11924328765793807|
|04/05/2018   |3966                |0.10812045698476963|
|06/26/2014   |3660                |0.0997783339798933 |
|01/20/2017   |3498                |0.0953619159184882 |
|04/10/2018   |3498                |0.0953619159184882 |
|04/24/2018   |3456                |0.09421691864330908|
|04/04/2018   |3318                |0.09045478473914917|
|09/13/2017   |3270                |0.08914621642465877|
|04/11/2018   |3186                |0.08685622187430056|
|04/06/2018   |3174                |0.08652907979567796|
|04/03/2018   |3126                |0.08522051148118756|
|03/06/2018   |3107                |0.08470253652336844|
|02/12/2018   |3090            

In [15]:
spark.sql("SELECT date_sent_to_company,COUNT(date_sent_to_company) Number_of_complaints, COUNT(*)/3668131*100 Percent_of_Total  FROM combined_dataset GROUP BY date_sent_to_company ORDER BY Number_of_complaints DESC").show(truncate=False)


+--------------------+--------------------+-------------------+
|date_sent_to_company|Number_of_complaints|Percent_of_Total   |
+--------------------+--------------------+-------------------+
|09/08/2017          |6930                |0.18892455040455208|
|09/09/2017          |5874                |0.16013604748576318|
|03/12/2014          |3738                |0.10190475749094022|
|01/13/2014          |3612                |0.0984697656654029 |
|04/10/2018          |3576                |0.0974883394295351 |
|02/07/2014          |3408                |0.09290835032881868|
|02/10/2014          |3408                |0.09290835032881868|
|04/24/2018          |3366                |0.09176335305363958|
|02/24/2014          |3336                |0.09094549785708309|
|01/19/2017          |3294                |0.08980050058190397|
|02/12/2018          |3216                |0.08767407707085706|
|01/20/2017          |3198                |0.08718336395292317|
|04/04/2018          |3174              

In [16]:
spark.sql("SELECT issue,COUNT(issue)  Number_of_complaints, COUNT(*)/3668131*100 Percent_of_Total FROM combined_dataset GROUP BY issue ORDER BY Number_of_complaints DESC").show(truncate=False)


+--------------------------------------------------------------------------------+--------------------+------------------+
|issue                                                                           |Number_of_complaints|Percent_of_Total  |
+--------------------------------------------------------------------------------+--------------------+------------------+
|Loan modification,collection,foreclosure                                        |409024              |11.150746797210896|
|Incorrect information on your report                                            |362805              |9.890731819556063 |
|Incorrect information on credit report                                          |341880              |9.320277819957901 |
|Loan servicing, payments, escrow account                                        |243839              |6.647499775771366 |
|Cont'd attempts collect debt not owed                                           |179055              |4.881368740647485 |
|Account opening

In [17]:
spark.sql("SELECT sub_issue,COUNT(sub_issue) Number_of_complaints, COUNT(*)/3668131*100 Percent_of_Total  FROM combined_dataset GROUP BY sub_issue ORDER BY Number_of_complaints DESC").show(truncate=False)


+--------------------------------------------------------+--------------------+------------------+
|sub_issue                                               |Number_of_complaints|Percent_of_Total  |
+--------------------------------------------------------+--------------------+------------------+
|                                                        |1650049             |44.98337164076201 |
|Information belongs to someone else                     |167879              |4.576690418090303 |
|Account status                                          |133008              |3.6260427994529096|
|Debt is not mine                                        |109924              |2.9967304875425658|
|Information is not mine                                 |107940              |2.942642997210296 |
|Their investigation did not fix an error on your report |88248               |2.405802846190608 |
|Not given enough info to verify debt                    |75066               |2.0464372728236806|
|Account s

In [18]:
spark.sql("SELECT state,COUNT(state)  Number_of_complaints, COUNT(*)/3668131*100 Percent_of_Total FROM combined_dataset GROUP BY state ORDER BY Number_of_complaints DESC").show(truncate=False)


+-----+--------------------+------------------+
|state|Number_of_complaints|Percent_of_Total  |
+-----+--------------------+------------------+
|CA   |633104              |17.259579878690264|
|FL   |399144              |10.881399819144955|
|TX   |342660              |9.341542055068372 |
|NY   |297131              |8.100337746934338 |
|GA   |220709              |6.016933419226303 |
|IL   |168828              |4.602561904141373 |
|MD   |126030              |3.4358096807338665|
|NC   |124488              |3.3937719236308626|
|VA   |118784              |3.2382703889255864|
|PA   |116430              |3.174096017835786 |
|OH   |113862              |3.1040876130105493|
|MI   |91872               |2.504599753934633 |
|AZ   |90497               |2.4671147240924602|
|WA   |81816               |2.2304546920488937|
|CO   |63936               |1.743012994901218 |
|TN   |59202               |1.613955444884602 |
|SC   |56196               |1.5320063541896405|
|NV   |53250               |1.4516929738

In [19]:
spark.sql("SELECT submitted_via,COUNT(submitted_via) Number_of_complaints, COUNT(*)/3668131*100 Percent_of_Total  FROM combined_dataset GROUP BY submitted_via ORDER BY Number_of_complaints DESC").show(truncate=False)



+-------------+--------------------+-------------------+
|submitted_via|Number_of_complaints|Percent_of_Total   |
+-------------+--------------------+-------------------+
|Web          |2312929             |63.05470006387449  |
|Referral     |706430              |19.258581550113668 |
|Postal mail  |291352              |7.9427915742376705 |
|Phone        |275430              |7.50872855958525   |
|Fax          |80640               |2.1983947683438787 |
|Email        |1350                |0.03680348384504261|
+-------------+--------------------+-------------------+

In [20]:
spark.sql("SELECT timely_response,COUNT(timely_response) Number_of_complaints, COUNT(*)/3668131*100 Percent_of_Total  FROM combined_dataset GROUP BY timely_response ORDER BY Number_of_complaints DESC").show(truncate=False)


+---------------+--------------------+-----------------+
|timely_response|Number_of_complaints|Percent_of_Total |
+---------------+--------------------+-----------------+
|Yes            |3582859             |97.67532838930781|
|No             |85272               |2.324671610692203|
+---------------+--------------------+-----------------+

In [21]:
spark.sql("SELECT tags,COUNT(tags) Number_of_complaints, COUNT(*)/3668131*100 Percent_of_Total  FROM combined_dataset GROUP BY tags ORDER BY Number_of_complaints DESC").show(truncate=False)


+-----------------------------+--------------------+------------------+
|tags                         |Number_of_complaints|Percent_of_Total  |
+-----------------------------+--------------------+------------------+
|                             |3228236             |88.00765294369258 |
|Older American               |211996              |5.77940100830641  |
|Servicemember                |193280              |5.269168413014693 |
|Older American, Servicemember|34619               |0.9437776349863187|
+-----------------------------+--------------------+------------------+

In [22]:
spark.sql("SELECT zip_code,COUNT(zip_code) Number_of_complaints, COUNT(zip_code)/3668131*100 Percent_of_Total FROM combined_dataset GROUP BY zip_code ORDER BY Number_of_complaints DESC").show(truncate=False)


+--------+--------------------+-------------------+
|zip_code|Number_of_complaints|Percent_of_Total   |
+--------+--------------------+-------------------+
|48382   |5670                |0.15457463214917896|
|30349   |5172                |0.14099823588634103|
|33071   |4428                |0.12071542701173976|
|28269   |4314                |0.11760757726482507|
|33173   |3984                |0.10861117010270353|
|60620   |3666                |0.09994190501920461|
|30331   |3666                |0.09994190501920461|
|33025   |3456                |0.09421691864330908|
|30058   |3432                |0.09356263448606388|
|30281   |3384                |0.09225406617157349|
|30034   |3288                |0.08963692954259267|
|20774   |3270                |0.08914621642465877|
|20744   |3264                |0.08898264538534748|
|90047   |3258                |0.08881907434603617|
|11375   |3246                |0.08849193226741356|
|30039   |3246                |0.08849193226741356|
|28216   |32

In [23]:
spark.sql("SELECT c2.sub_product, COUNT(c2.sub_product) Number_of_complaints, COUNT(c2.sub_product)/3668131*100 Percent_of_Total FROM combined_dataset c2 GROUP BY c2.sub_product ORDER BY Number_of_complaints DESC").show(50,truncate=False)


+------------------------------------------+--------------------+-------------------+
|sub_product                               |Number_of_complaints|Percent_of_Total   |
+------------------------------------------+--------------------+-------------------+
|                                          |747242              |20.371191759509134 |
|Credit reporting                          |565326              |15.411826894950046 |
|Other mortgage                            |336071              |9.161913792064679  |
|Checking account                          |277317              |7.560171651448654  |
|Conventional fixed mortgage               |217098              |5.918490915400786  |
|I do not know                             |142432              |3.882958378531192  |
|Other (i.e. phone, health club, etc.)     |135911              |3.7051839206396937 |
|Conventional adjustable mortgage (ARM)    |86358               |2.354277968807548  |
|FHA mortgage                              |86177     

### Cross-frequency of some of the fields (group by multiple fields)


In [183]:
spark.sql("SELECT t1.product, c1.sub_product, COUNT(c1.sub_product)  Number_of_complaints, COUNT(*)/3668131*100 Percent_of_Total FROM combined_dataset c1 CROSS JOIN (SELECT DISTINCT c2.product FROM combined_dataset c2)t1 ON t1.product = c1.product GROUP BY t1.product, c1.sub_product ORDER BY product, Number_of_Complaints DESC").show(20, truncate = False)


+---------------------------+------------------------------------------+--------------------+---------------------+
|product                    |sub_product                               |Number_of_complaints|Percent_of_Total     |
+---------------------------+------------------------------------------+--------------------+---------------------+
|Bank account or service    |Checking account                          |194806              |5.310769980679534    |
|Bank account or service    |Other bank product/service                |56207               |1.5323062344283778   |
|Bank account or service    |Savings account                           |17183               |0.46844019474767945  |
|Bank account or service    |(CD) Certificate of deposit               |11520               |0.314056395477697    |
|Bank account or service    |Cashing a check without an account        |1806                |0.04923488283270145  |
|Checking or savings account|Checking account                          |

In [25]:
spark.sql("SELECT t1.issue, c1.sub_issue, COUNT(c1.sub_issue) Number_of_complaints, COUNT(*)/3668131*100 Percent_of_Total FROM combined_dataset c1 CROSS JOIN (SELECT DISTINCT c2.issue FROM combined_dataset c2)t1 ON t1.issue = c1.issue GROUP BY t1.issue, c1.sub_issue ORDER BY issue, Number_of_Complaints DESC").show()



+--------------------+--------------------+--------------------+--------------------+
|               issue|           sub_issue|Number_of_complaints|    Percent_of_Total|
+--------------------+--------------------+--------------------+--------------------+
|APR or interest rate|                    |               17604| 0.47991742933935566|
|Account opening, ...|                    |              123976|   3.379813861609632|
|Account terms and...|                    |                1662| 0.04530917788923024|
|        Adding money|                    |                 450|0.012267827948347538|
|         Advertising|Confusing or misl...|                  72|0.001962852471735...|
|         Advertising|Changes in terms ...|                  48|0.001308568314490404|
|Advertising and m...|                    |                8532|  0.2325980179006693|
|Advertising and m...|Didn't receive ad...|                3180| 0.08669265083498927|
|Advertising and m...|Confusing or misl...|           

### Finding missing (NULL) values for each column

In [26]:
spark.sql("SELECT count(*) FROM combined_dataset WHERE complaint_id IS NULL OR complaint_id = ''").show()

+--------+
|count(1)|
+--------+
|       0|
+--------+

In [27]:
spark.sql("SELECT count(*) FROM combined_dataset WHERE date_received IS NULL OR date_received = ''").show()


+--------+
|count(1)|
+--------+
|       0|
+--------+

In [28]:
spark.sql("SELECT count(*) FROM combined_dataset WHERE product IS NULL OR product = ''").show()


+--------+
|count(1)|
+--------+
|       0|
+--------+

In [29]:
spark.sql("SELECT count(*) FROM combined_dataset WHERE sub_product IS NULL OR sub_product = ''").show()


+--------+
|count(1)|
+--------+
|  747242|
+--------+

In [30]:
spark.sql("SELECT count(*) FROM combined_dataset WHERE issue IS NULL OR issue = ''").show()


+--------+
|count(1)|
+--------+
|       0|
+--------+

In [31]:
spark.sql("SELECT count(*) FROM combined_dataset WHERE sub_issue IS NULL OR sub_issue = ''").show()


+--------+
|count(1)|
+--------+
| 1650049|
+--------+

In [32]:
spark.sql("SELECT count(*) FROM combined_dataset WHERE company_public_response IS NULL OR company_public_response = ''").show()


+--------+
|count(1)|
+--------+
| 2541449|
+--------+

In [33]:
spark.sql("SELECT count(*) FROM combined_dataset WHERE tags IS NULL OR tags = ''").show()


+--------+
|count(1)|
+--------+
| 3228236|
+--------+

In [34]:
spark.sql("SELECT count(*) FROM combined_dataset WHERE consumer_consent_provided IS NULL OR consumer_consent_provided = ''").show()


+--------+
|count(1)|
+--------+
|   38034|
+--------+

In [35]:
spark.sql("SELECT count(*) FROM combined_dataset WHERE submitted_via IS NULL OR submitted_via = ''").show()


+--------+
|count(1)|
+--------+
|       0|
+--------+

In [36]:
spark.sql("SELECT count(*) FROM combined_dataset WHERE date_sent_to_company IS NULL OR date_sent_to_company = ''").show()


+--------+
|count(1)|
+--------+
|       0|
+--------+

In [37]:
spark.sql("SELECT count(*) FROM combined_dataset WHERE company_response IS NULL OR company_response = ''").show()


+--------+
|count(1)|
+--------+
|      12|
+--------+

In [38]:
spark.sql("SELECT count(*) FROM combined_dataset WHERE timely_response IS NULL OR timely_response = ''").show()


+--------+
|count(1)|
+--------+
|       0|
+--------+

In [39]:
spark.sql("SELECT count(*) FROM combined_dataset WHERE consumer_disputed IS NULL OR consumer_disputed = ''").show()


+--------+
|count(1)|
+--------+
|       0|
+--------+

In [40]:
spark.sql("SELECT count(*) FROM combined_dataset WHERE state IS NULL OR state = ''").show()


+--------+
|count(1)|
+--------+
|      30|
+--------+

In [41]:
spark.sql("SELECT count(*) FROM combined_dataset WHERE Average_Complaints IS NULL OR Average_Complaints = ''").show()


+--------+
|count(1)|
+--------+
|       0|
+--------+

In [42]:
spark.sql("SELECT count(*) FROM combined_dataset WHERE Average_Income IS NULL OR Average_Income = ''").show()


+--------+
|count(1)|
+--------+
|       0|
+--------+

In [43]:
spark.sql("SELECT count(*) FROM combined_dataset WHERE population IS NULL OR population = ''").show()


+--------+
|count(1)|
+--------+
|       0|
+--------+

In [44]:
spark.sql("SELECT count(*) FROM combined_dataset WHERE Total_Income IS NULL OR Total_Income = ''").show()


+--------+
|count(1)|
+--------+
|       0|
+--------+

### Distinct values for each column. Showing only top 20

In [45]:
finalDF.distinct().count()

3666438

In [46]:
from pyspark.sql.functions import countDistinct, approx_count_distinct





In [47]:
print('Count distinct values:')

Count distinct values:

In [48]:
finalDF.select(countDistinct("product").alias("Distinct products")).show()

+-----------------+
|Distinct products|
+-----------------+
|               18|
+-----------------+

In [49]:
finalDF.select(countDistinct("sub_product").alias("Distinct sub_products")).show()

+---------------------+
|Distinct sub_products|
+---------------------+
|                   77|
+---------------------+

In [50]:
finalDF.select(countDistinct("issue").alias("Distinct issues")).show()

+---------------+
|Distinct issues|
+---------------+
|            167|
+---------------+

In [51]:
finalDF.select(countDistinct("sub_issue").alias("Distinct sub_issue")).show()

+------------------+
|Distinct sub_issue|
+------------------+
|               218|
+------------------+

In [52]:
finalDF.select(countDistinct("company_public_response").alias("Distinct company_public_responses")).show()

+---------------------------------+
|Distinct company_public_responses|
+---------------------------------+
|                               11|
+---------------------------------+

In [53]:
finalDF.select(countDistinct("tags").alias("Distinct tags")).show()

+-------------+
|Distinct tags|
+-------------+
|            4|
+-------------+

In [54]:
finalDF.select(countDistinct("consumer_consent_provided").alias("Distinct consumer_consent_provided")).show()

+----------------------------------+
|Distinct consumer_consent_provided|
+----------------------------------+
|                                 6|
+----------------------------------+

In [55]:
finalDF.select(countDistinct("submitted_via").alias("Distinct submitted_via")).show()

+----------------------+
|Distinct submitted_via|
+----------------------+
|                     6|
+----------------------+

In [56]:
finalDF.select(countDistinct("company_response").alias("Distinct company_response")).show()

+-------------------------+
|Distinct company_response|
+-------------------------+
|                        9|
+-------------------------+

In [57]:
finalDF.select(countDistinct("timely_response").alias("Distinct timely_response")).show()

+------------------------+
|Distinct timely_response|
+------------------------+
|                       2|
+------------------------+

In [58]:
finalDF.select(countDistinct("consumer_disputed").alias("Distinct consumer_disputed")).show()

+--------------------------+
|Distinct consumer_disputed|
+--------------------------+
|                         3|
+--------------------------+

In [59]:
finalDF.select(countDistinct("state").alias("Distinct state")).show()

+--------------+
|Distinct state|
+--------------+
|            64|
+--------------+

# b)  In my analysis I will compare all the factors(columns) for two companies I would select. For doing a competitive analysis for two organizations

Checking for two companies having similar issues

Total Complaints for Company where Product is credit

In [13]:
%%sql -o query1 
SELECT company, Num_of_complaints, DENSE_RANK() OVER(ORDER BY Num_of_complaints DESC) Rank_No_of_complaints FROM (SELECT company, COUNT(*) Num_of_complaints FROM combined_dataset WHERE product like '%credit%' GROUP BY company)SUB


Output()

Creating another dataframe filtering for top two companies based on the number of complaints for the Credit card product

In [18]:
df_experian_equi = spark.sql("SELECT * FROM combined_dataset WHERE company = 'EQUIFAX, INC.' OR company = 'Experian Information Solutions Inc.'")

In [19]:
df_experian_equi.createOrReplaceTempView("experian_equi")
spark.sql("show tables").show()

+--------+----------------+-----------+
|database|       tableName|isTemporary|
+--------+----------------+-----------+
|        |combined_dataset|       true|
|        |            corr|       true|
|        |   experian_equi|       true|
+--------+----------------+-----------+

#### Top products

In [20]:
%%sql -o query2
SELECT product, COUNT(*) Num_of_complaints FROM experian_equi WHERE company = 'EQUIFAX, INC.' \
GROUP BY product ORDER BY product

Output()

In [21]:
%%sql -o query2
SELECT product, COUNT(*) Num_of_complaints FROM experian_equi WHERE company = 'Experian Information Solutions Inc.' \
GROUP BY product  ORDER BY product

Output()

#### Top sub-products

In [22]:
%%sql -o query2
SELECT sub_product, COUNT(*) Num_of_complaints FROM experian_equi WHERE company = 'EQUIFAX, INC.' \
GROUP BY sub_product ORDER BY sub_product

Output()

In [23]:
%%sql -o query2
SELECT sub_product, COUNT(*) Num_of_complaints FROM experian_equi WHERE company = 'Experian Information Solutions Inc.' GROUP BY sub_product  ORDER BY sub_product

Output()

#### Main issues for each of the two companies

In [24]:
%%sql -o query2
SELECT issue, COUNT(*) Num_of_complaints FROM experian_equi WHERE company = 'EQUIFAX, INC.' GROUP BY issue ORDER BY issue

Output()

In [25]:
%%sql -o query2
SELECT issue, COUNT(*) Num_of_complaints FROM experian_equi WHERE company = 'Experian Information Solutions Inc.' GROUP BY issue ORDER BY issue

Output()

#### Main sub-issues for each of the two companies

In [26]:
%%sql -o query2
SELECT sub_issue, COUNT(*) Num_of_complaints FROM experian_equi WHERE company = 'EQUIFAX, INC.' GROUP BY sub_issue ORDER BY sub_issue

Output()

In [27]:
%%sql -o query2
SELECT sub_issue, COUNT(*) Num_of_complaints FROM experian_equi WHERE company = 'Experian Information Solutions Inc.' GROUP BY sub_issue ORDER BY sub_issue

Output()

#### Date analysis

In [28]:
df_experian_equi_date = df_experian_equi.withColumn('Clean_date_received',regexp_replace(col("date_received"), "/", "-")).withColumn('Clean_date_sent_to_company',regexp_replace(col("date_sent_to_company"), "/", "-"))


In [29]:
from pyspark.sql import functions as F
df_year_month = df_experian_equi_date.withColumn('Year_date_received', F.split(df_experian_equi_date.Clean_date_received, '-')[2]).withColumn('Month_date_received', F.split(df_experian_equi_date.Clean_date_received, '-')[0]).withColumn('DATE_date_received', F.split(df_experian_equi_date.Clean_date_received, '-')[1])
df_year_month.head()
#df_year_month.groupBy("Year","Month").count().orderBy("Year","Month").show(1)

Row(date_received='10/26/2018', product='Credit reporting, credit repair services, or other personal consumer reports', sub_product='Credit reporting', issue='Incorrect information on your report', sub_issue='Account status incorrect', company_public_response='Company has responded to the consumer and the CFPB and chooses not to provide a public response', company='Experian Information Solutions Inc.', state='NY', zip_code='10075', tags='', consumer_consent_provided='Consent not provided', submitted_via='Web', date_sent_to_company='10/26/2018', company_response='Closed with explanation', timely_response='Yes', consumer_disputed='N/A', complaint_id='3057600', population=26121, Total_Income='5179299', NUMDEP='2770', zipcode_agg='10075', Number_of_Complaints=558, zipcode_avgcc='10075', Average_Complaints=214, Average_Income=994602, Clean_date_received='10-26-2018', Clean_date_sent_to_company='10-26-2018', Year_date_received='2018', Month_date_received='10', DATE_date_received='26')

In [30]:
df_year_month.createOrReplaceTempView("experian_equi_date_year_month")
#df_experian_equi_date.printSchema()

Finding top year for both the companies

In [31]:
%%sql -o query2

select Year_date_received, COUNT(*) Num_of_complaints FROM experian_equi_date_year_month WHERE company = 'EQUIFAX, INC.' \
GROUP BY Year_date_received ORDER BY Year_date_received

Output()

In [61]:
%%sql -o query2

select Year_date_received, COUNT(*) Num_of_complaints FROM experian_equi_date_year_month \
WHERE company = 'Experian Information Solutions Inc.' GROUP BY Year_date_received ORDER BY Year_date_received

Output()

Plotting the month wise data for 2018 for both companies

In [33]:
%%sql -o query2

select Month_date_received, COUNT(*) Num_of_complaints FROM experian_equi_date_year_month \
WHERE company = 'EQUIFAX, INC.' AND Year_date_received = '2018' \
GROUP BY Month_date_received ORDER BY Month_date_received

Output()

In [34]:
%%sql -o query2

select Month_date_received, COUNT(*) Num_of_complaints FROM experian_equi_date_year_month \
WHERE company = 'Experian Information Solutions Inc.' AND Year_date_received = '2018' \
GROUP BY Month_date_received ORDER BY Month_date_received

Output()

In [35]:
%%sql -o query2

select DATE_date_received, COUNT(*) Num_of_complaints FROM experian_equi_date_year_month \
WHERE company = 'EQUIFAX, INC.' AND Year_date_received = '2018' AND Month_date_received = '04' \
GROUP BY DATE_date_received ORDER BY DATE_date_received

Output()

In [36]:
%%sql -o query2

select DATE_date_received, COUNT(*) Num_of_complaints FROM experian_equi_date_year_month \
WHERE company = 'Experian Information Solutions Inc.' AND Year_date_received = '2018' AND Month_date_received = '04' \
GROUP BY DATE_date_received ORDER BY DATE_date_received

Output()

company_public_response for both companies

In [37]:
%%sql -o query2

select company_public_response, COUNT(*) Num_of_complaints FROM experian_equi_date_year_month \
WHERE company = 'EQUIFAX, INC.' \
GROUP BY company_public_response ORDER BY company_public_response

Output()

In [38]:
%%sql -o query2

select company_public_response, COUNT(*) Num_of_complaints FROM experian_equi_date_year_month \
WHERE company = 'Experian Information Solutions Inc.' \
GROUP BY company_public_response ORDER BY company_public_response

Output()

tags                     

In [39]:
%%sql -o query2

select tags, COUNT(*) Num_of_complaints FROM experian_equi_date_year_month \
WHERE company = 'EQUIFAX, INC.' \
GROUP BY tags ORDER BY tags

Output()

In [40]:
%%sql -o query2

select tags, COUNT(*) Num_of_complaints FROM experian_equi_date_year_month \
WHERE company = 'Experian Information Solutions Inc.' \
GROUP BY tags ORDER BY tags

Output()

consumer_consent_provided

In [41]:
%%sql -o query2

select consumer_consent_provided, COUNT(*) Num_of_complaints FROM experian_equi_date_year_month \
WHERE company = 'EQUIFAX, INC.' \
GROUP BY consumer_consent_provided ORDER BY consumer_consent_provided

Output()

In [42]:
%%sql -o query2

select consumer_consent_provided, COUNT(*) Num_of_complaints FROM experian_equi_date_year_month \
WHERE company = 'Experian Information Solutions Inc.' \
GROUP BY consumer_consent_provided ORDER BY consumer_consent_provided

Output()

submitted_via            

In [43]:
%%sql -o query2

select submitted_via, COUNT(*) Num_of_complaints FROM experian_equi_date_year_month \
WHERE company = 'EQUIFAX, INC.' \
GROUP BY submitted_via ORDER BY submitted_via

Output()

In [44]:
%%sql -o query2

select submitted_via, COUNT(*) Num_of_complaints FROM experian_equi_date_year_month \
WHERE company = 'Experian Information Solutions Inc.' \
GROUP BY submitted_via ORDER BY submitted_via

Output()

company_response         

In [45]:
%%sql -o query2

select company_response, COUNT(*) Num_of_complaints FROM experian_equi_date_year_month \
WHERE company = 'EQUIFAX, INC.' \
GROUP BY company_response ORDER BY company_response


Output()

In [46]:
%%sql -o query2

select company_response, COUNT(*) Num_of_complaints FROM experian_equi_date_year_month \
WHERE company = 'Experian Information Solutions Inc.' \
GROUP BY company_response ORDER BY company_response

Output()

timely_response          

In [47]:
%%sql -o query2

select timely_response, COUNT(*) Num_of_complaints FROM experian_equi_date_year_month \
WHERE company = 'EQUIFAX, INC.' \
GROUP BY timely_response ORDER BY timely_response

Output()

In [48]:
%%sql -o query2

select timely_response, COUNT(*) Num_of_complaints FROM experian_equi_date_year_month \
WHERE company = 'Experian Information Solutions Inc.' \
GROUP BY timely_response ORDER BY timely_response

Output()

consumer_disputed        

In [49]:
%%sql -o query2

select consumer_disputed, COUNT(*) Num_of_complaints FROM experian_equi_date_year_month \
WHERE company = 'EQUIFAX, INC.' \
GROUP BY consumer_disputed ORDER BY consumer_disputed

Output()

In [50]:
%%sql -o query2

select consumer_disputed, COUNT(*) Num_of_complaints FROM experian_equi_date_year_month \
WHERE company = 'Experian Information Solutions Inc.' \
GROUP BY consumer_disputed ORDER BY consumer_disputed

Output()

In [51]:
%%sql -o query2

select distinct zip_code, Average_Complaints FROM experian_equi_date_year_month \
WHERE company = 'EQUIFAX, INC.' \
ORDER BY Average_Complaints DESC, zip_code

Output()

In [52]:
%%sql -o query2

select distinct zip_code, Average_Complaints FROM experian_equi_date_year_month \
WHERE company = 'Experian Information Solutions Inc.' \
ORDER BY Average_Complaints DESC, zip_code

Output()

## c)  To get average number of complaints in each state 

State-wise analysis

Top state by total number of complaints

In [53]:
%%sql -o query2

select state, COUNT(*) Total_Number_of_complaints FROM experian_equi_date_year_month \
WHERE company = 'EQUIFAX, INC.' \
GROUP BY state
ORDER BY state

Output()

In [54]:
%%sql -o query2

select state,  COUNT(*) Total_Number_of_complaints  FROM experian_equi_date_year_month \
WHERE company = 'Experian Information Solutions Inc.' \
GROUP BY state
ORDER BY state

Output()

Top states by average complaints. Calculated the average by following method:

divided the total number of complaints for a zipcode by the population of that zipcode
scaling factor of 10000

In [55]:
%%sql -o query2

select state, SUM(Average_Complaints) Total_avg_complaints FROM experian_equi_date_year_month \
WHERE company = 'EQUIFAX, INC.' \
GROUP BY state
ORDER BY state

Output()

In [56]:
%%sql -o query2

select state, SUM(Average_Complaints) Total_avg_complaints  FROM experian_equi_date_year_month \
WHERE company = 'Experian Information Solutions Inc.' \
GROUP BY state
ORDER BY state

Output()

# d)  I would also like to check if there more complaints from the area having lower average income? 

In [57]:
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler


In [58]:
from pyspark.sql.functions import countDistinct, approx_count_distinct, count, sum, mean, round
corr_DF = finalDF.groupBy("zip_code").agg(round(mean("Number_of_Complaints"),3).alias("Average_Number_of_Complaints"),
                         round(mean("average_income"),3).alias("average_income"))

In [8]:
corr_DF.show(2)

+--------+----------------------------+--------------+
|zip_code|Average_Number_of_Complaints|average_income|
+--------+----------------------------+--------------+
|   11563|                       540.0|      167009.0|
|   11722|                       756.0|      136987.0|
+--------+----------------------------+--------------+
only showing top 2 rows

In [168]:
#corr_avg_inc_DF = finalDF.select('Number_of_Complaints', 'average_income')

In [169]:
#corr_avg_inc_DF = corr_avg_inc_DF.dropDuplicates(['Number_of_Complaints', 'average_income'])

In [170]:
#corr_avg_inc_DF.show(2)

In [159]:
#spark.sql("drop table corr_avg_inc")

DataFrame[]

In [59]:
corr_DF.createOrReplaceTempView("corr")
spark.sql("show tables").show(truncate=False)
spark.sql("desc corr").show(truncate=False)

+--------+-----------------------------+-----------+
|database|tableName                    |isTemporary|
+--------+-----------------------------+-----------+
|        |combined_dataset             |true       |
|        |corr                         |true       |
|        |experian_equi                |true       |
|        |experian_equi_date_year_month|true       |
+--------+-----------------------------+-----------+

+----------------------------+---------+-------+
|col_name                    |data_type|comment|
+----------------------------+---------+-------+
|zip_code                    |string   |null   |
|Average_Number_of_Complaints|double   |null   |
|average_income              |double   |null   |
+----------------------------+---------+-------+

In [162]:
spark.sql("SELECT distinct Number_of_Complaints,average_income FROM combined_dataset ").show(2)

+--------------------+--------------+
|Number_of_Complaints|average_income|
+--------------------+--------------+
|                1242|        211485|
|                  48|         14379|
+--------------------+--------------+
only showing top 2 rows

In [60]:
%%sql -o query2

SELECT average_income, Average_Number_of_Complaints FROM corr                    

Output()

In [179]:
corr_DF = corr_DF.drop('zip_code')

In [180]:
# convert to vector column first
vector_col = "corr_features"
assembler = VectorAssembler(inputCols=corr_DF.columns, outputCol=vector_col)
df_vector = assembler.transform(corr_DF).select(vector_col)

In [181]:
# get correlation matrix

Pearson_matrix  = Correlation.corr(df_vector, vector_col).head()
print("Pearson correlation matrix:\n" + str(Pearson_matrix[0]))

Pearson correlation matrix:
DenseMatrix([[1.        , 0.60189822],
             [0.60189822, 1.        ]])

In [182]:
spearman_matrix = Correlation.corr(df_vector, vector_col, "spearman").head()
print("Spearman correlation matrix:\n" + str(spearman_matrix[0]))

Spearman correlation matrix:
DenseMatrix([[1.        , 0.80828985],
             [0.80828985, 1.        ]])